# the cryptopals crypto challenges Set 1

## Convert hex to base64

The string:
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d

Should produce:
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t

So go ahead and make that happen. You'll need to use this code for the rest of the exercises.

### Cryptopals Rule!!
Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

In [1]:
import base64


hexstring = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
base64string = 'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t'

def hex_to_base46(hex_string: str) -> str:
    raw_bytes = bytes.fromhex(hex_string)
    return base64.b64encode(raw_bytes).decode()

res = hex_to_base46(hexstring)
print(res)
print(f'{res == base64string}')

SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
True


## Fixed XOR

Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c

... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965

... should produce:

746865206b696420646f6e277420706c6179

In [2]:
buffer_1 = '1c0111001f010100061a024b53535009181c'
buffer_2 ='686974207468652062756c6c277320657965'
expected_result = '746865206b696420646f6e277420706c6179'

def xor_hex_strings(hex_string_1: str, hex_string_2: str) -> str:
    if len(hex_string_1) != len(hex_string_2):
        raise ValueError("Input hex strings must have the same length.")

    raw_bytes_1 = bytes.fromhex(hex_string_1)
    raw_bytes_2 = bytes.fromhex(hex_string_2)
    res = bytes(x ^ y for x ,y  in zip(raw_bytes_1, raw_bytes_2))
    return res.hex()

res = xor_hex_strings(buffer_1, buffer_2)
print(res)
print(expected_result == res)

746865206b696420646f6e277420706c6179
True


## Single-byte XOR cipher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.

In [3]:


hex_string = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

english_freq = {
    'a': 8.167, 'b': 1.492, 'c': 2.782, 'd': 4.253, 'e': 12.702,
    'f': 2.228, 'g': 2.015, 'h': 6.094, 'i': 6.966, 'j': 0.153,
    'k': 0.772, 'l': 4.025, 'm': 2.406, 'n': 6.749, 'o': 7.507,
    'p': 1.929, 'q': 0.095, 'r': 5.987, 's': 6.327, 't': 9.056,
    'u': 2.758, 'v': 0.978, 'w': 2.360, 'x': 0.150, 'y': 1.974,
    'z': 0.074, ' ': 13.000  # Including space as it's common in English text
}

def score_text_on_character_frequency( text: str ) -> float:
    return sum( english_freq.get(char, 0) for char in text )

def single_byte_xor_decrypt( text_bytes: bytes, key: int ) -> bytes:
    return bytes( byte ^ key for byte in text_bytes )

def guess_xor_key(raw_byes: str) -> tuple[str, int, float]:
    best_text = ''
    best_key = 0
    best_score = 0

    for i in range(256):
        try:
            decrypted = single_byte_xor_decrypt(raw_byes, i)
            text = decrypted.decode()
            score = score_text_on_character_frequency(text)
            if score > best_score:
                best_score = score
                best_text = text
                best_key = i
        except UnicodeDecodeError:
            continue
    return best_text, best_key, best_score


text, key, score = guess_xor_key(bytes.fromhex(hex_string))
print(text)
print(key)
print(key.to_bytes().decode())

Cooking MC's like a pound of bacon
88
X




## Detect single-character XOR

One of the 60-character strings in this 4.txt has been encrypted by single-character XOR.

Find it.

In [4]:
with open('4.txt') as file:
    lines = [line.rstrip() for line in file]

decoded = [guess_xor_key(bytes.fromhex(line)) for line in lines]
sorted_decoded = sorted(decoded, key=lambda x: x[2], reverse=True)
print(sorted_decoded[0][0])

Now that the party is jumping



## Implement repeating-key XOR

Here is the opening stanza of an important work of the English language:

Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
Encrypt it, under the key "ICE", using repeating-key XOR.

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f

Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.

In [5]:
plain_text = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
expected_cyphertext ="0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"
encryption_key = 'ICE'


def repeating_key_xor_encrypter( text_bytes: bytes, key_bytes: bytes ) -> str:
    res = []
    for i in range(len(text_bytes)):
        tb = text_bytes[i]
        # modulo needed to repeat the key
        kb = key_bytes[(i % len(key_bytes))]
        encoded_char = tb ^ kb
        # XOR results are represented in hex to provide a clear, consistent, readable format.
        res.append(f'{encoded_char:02x}')
    return ''.join(res)


res = repeating_key_xor_encrypter(plain_text.encode(), encryption_key.encode())
print(res)
print(res == expected_cyphertext)

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f
True


## Break repeating-key XOR

It is officially on, now.
This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.

6.txt has been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.

2. Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
`this is a test`
and
`wokka wokka!!!`
is `'37'`. Make sure your code agrees before you proceed.

In [6]:
test_input1 ='this is a test'
test_input2 ='wokka wokka!!!'
expected_test_result = 37

def compute_hamming_distance(bytes_1: bytes, bytes_2: bytes) -> int:
    if len(bytes_1) != len(bytes_2):
        raise ValueError("Strings must be of equal length")
    
    distance = 0
    for b1, b2 in zip(bytes_1, bytes_2):
        # XOR the bytes and count the number of 1 bits
        xor_result = b1 ^ b2
        distance += bin(xor_result).count('1')

    return distance

res = compute_hamming_distance(test_input1.encode(), test_input2.encode())
print(res)
print(res == expected_test_result)

37
True


3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.

4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.

In [7]:
from math import inf


with open('6.txt') as f:
    data = f.read()

data = data.replace('\n', '')
print(data)
original_bytes = base64.b64decode(data)
print(original_bytes)
print('-------------------')

def guess_key_lenght(encryptet_bytes: bytes) -> int:
    sum = 0
    MAX_LENGTHT = 40
    guessed_length = 1
    top_score = inf

    for i in range(1, MAX_LENGTHT + 1):
        rotated_string = encryptet_bytes[i:] + encryptet_bytes[:i]
        score = compute_hamming_distance(rotated_string, encryptet_bytes)
        sum += score
        if score < top_score:
            top_score = score 
            guessed_length = i
    print(f"average distance = {sum / MAX_LENGTHT}")
    return guessed_length, top_score

guessed_key_length, top_score = guess_key_lenght(original_bytes)
print(guessed_key_length, top_score)

HUIfTQsPAh9PE048GmllH0kcDk4TAQsHThsBFkU2AB4BSWQgVB0dQzNTTmVSBgBHVBwNRU0HBAxTEjwMHghJGgkRTxRMIRpHKwAFHUdZEQQJAGQmB1MANxYGDBoXQR0BUlQwXwAgEwoFR08SSAhFTmU+Fgk4RQYFCBpGB08fWXh+amI2DB0PQQ1IBlUaGwAdQnQEHgFJGgkRAlJ6f0kASDoAGhNJGk9FSA8dDVMEOgFSGQELQRMGAEwxX1NiFQYHCQdUCxdBFBZJeTM1CxsBBQ9GB08dTnhOSCdSBAcMRVhICEEATyBUCHQLHRlJAgAOFlwAUjBpZR9JAgJUAAELB04CEFMBJhAVTQIHAh9PG054MGk2UgoBCVQGBwlTTgIQUwg7EAYFSQ8PEE87ADpfRyscSWQzT1QCEFMaTwUWEXQMBk0PAg4DQ1JMPU4ALwtJDQhOFw0VVB1PDhxFXigLTRkBEgcKVVN4Tk9iBgELR1MdDAAAFwoFHww6Ql5NLgFBIg4cSTRWQWI1Bk9HKn47CE8BGwFTQjcEBx4MThUcDgYHKxpUKhdJGQZZVCFFVwcDBVMHMUV4LAcKQR0JUlk3TwAmHQdJEwATARNFTg5JFwQ5C15NHQYEGk94dzBDADsdHE4UVBUaDE5JTwgHRTkAUmc6AUETCgYAN1xGYlUKDxJTEUgsAA0ABwcXOwlSGQELQQcbE0c9GioWGgwcAgcHSAtPTgsAABY9C1VNCAINGxgXRHgwaWUfSQcJABkRRU8ZAUkDDTUWF01jOgkRTxVJKlZJJwFJHQYADUgRSAsWSR8KIgBSAAxOABoLUlQwW1RiGxpOCEtUYiROCk8gUwY1C1IJCAACEU8QRSxORTBSHQYGTlQJC1lOBAAXRTpCUh0FDxhUZXhzLFtHJ1JbTkoNVDEAQU4bARZFOwsXTRAPRlQYE042WwAuGxoaAk5UHAoAZCYdVBZ0ChQLSQMYVAcXQTwaUy1SBQsTAAAAAAAM

5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.

In [8]:
def split_bytes_into_chunks(data: bytes, chunk_size: int) -> list:
    return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

encrypted_junks = split_bytes_into_chunks(original_bytes, guessed_key_length)

test_data = b"abc123*%&+"
test_chunks = split_bytes_into_chunks(test_data, 3)

for chunk in test_chunks:
    print(f"{chunk} is {len(chunk)} bytes long")

b'abc' is 3 bytes long
b'123' is 3 bytes long
b'*%&' is 3 bytes long
b'+' is 1 bytes long


6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.

In [9]:
def transpose_chunks(
        chunks: list[bytes],
        fill: bool = False,
        fill_byte: bytes = b'\x00'
        ) -> list[bytes]:
    if len(chunks[-1]) < len(chunks[0]):
        if fill:
            chunks[-1] += fill_byte * (len(chunks[0]) - len(chunks[-1]))
        else:
            chunks = chunks[:-1]
    return [bytes(block) for block in zip(*chunks)]

print(transpose_chunks(test_chunks))
print(transpose_chunks(test_chunks, True))

transposed_blocks = transpose_chunks(encrypted_junks)

[b'a1*', b'b2%', b'c3&']
[b'a1*+', b'b2%\x00', b'c3&\x00']


7. Solve each block as if it was single-character XOR. You already have code to do this.

8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

In [10]:
def get_encryption_byte_per_block(encrypted_blocks: list[bytes]) -> list[int]:
    keys = []
    for block in encrypted_blocks:
        _, key, _ = guess_xor_key(block)
        keys.append(key)
    return keys

guessed_keyparts = get_encryption_byte_per_block(transposed_blocks)
key = bytes(guessed_keyparts).decode()
print(key)

Terminator X: Bring the noise


In [11]:
text = repeating_key_xor_encrypter(original_bytes, key.encode())
print(bytes.fromhex(text).decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 
